# 2.Data section

### 2.1 Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ 

### 2.2 Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'JCCJHCOKV0OOM2AGRVFNBUW5I4DTTWFLOVXA5KXN44YK3JTD' 
CLIENT_SECRET = 'WKO2TZCE3LTCEGYGRNWAWJHVMHODYH0HXQX0ELWWUK3L0T1E' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JCCJHCOKV0OOM2AGRVFNBUW5I4DTTWFLOVXA5KXN44YK3JTD
CLIENT_SECRET:WKO2TZCE3LTCEGYGRNWAWJHVMHODYH0HXQX0ELWWUK3L0T1E


### 2.3 Use geopy library to get the latitude and longitude values of Shanghai.
**In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent foursquare_agent, as shown below.**

In [5]:
address = 'Shanghai, China'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

31.2322758 121.4692071


### 2.4 Collect districts latitude and longitude data of Shanghai
**Shanghai is divided into 16 districts. In order to segement the districts and explore them, we will essentially need a dataset that contains the 16 districts that exist in each district as well as the the latitude and logitude coordinates of each district.**

**I searched online and found the link to the dataset. The data set inforamtion is from http://bigdata.cmmrmedia.com/.**


In [18]:
!wget -q -O 'shanghai.json' http://bigdata.cmmrmedia.com/echarts/china/json/shanghai.json
print('Data downloaded!')

Data downloaded!


**Load and explore the data**

In [19]:
with open('shanghai.json') as json_data:
    shanghai_data = json.load(json_data)

**Since all the relevant data is in the features key, which is basically a list of the districts. So, let's define a new variable that includes this data.**

In [20]:
district_data = shanghai_data['features']

**Let's take a look at the first item in this list.**

In [21]:
district_data[0]

{'id': '310101',
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': ['@@AHV@BDCPEPLENFHCJHFBNCAGBGXWFIBGCGMUKKCCcGCDGBALCPIAAPGR'],
  'encodeOffsets': [[124373, 31970]]},
 'properties': {'cp': [121.490317, 31.222771], 'name': '黄浦区', 'childNum': 1}}

**Tranform the data into a pandas dataframe
The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.**

In [22]:
# define the dataframe columns
column_names = ['District', 'Latitude', 'Longitude'] 

# instantiate the dataframe
Districts = pd.DataFrame(columns=column_names)

**Take a look at the empty dataframe to confirm that the columns are as intended.**

In [23]:
Districts.head() 

ImportError: cannot import name 'OrderedDict'

Empty DataFrame
Columns: [District, Latitude, Longitude]
Index: []

In [ ]:
Then let's loop through the data and fill the dataframe one row at a time.

In [16]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['properties']['cp']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    Districts = Districts.append({'District': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

NameError: name 'neighborhoods_data' is not defined

In [17]:
Districts

ImportError: cannot import name 'OrderedDict'

Empty DataFrame
Columns: [District, Latitude, Longitude]
Index: []

In [53]:
print('The dataframe has '+ str(Districts.shape[0]) +' districts.')

The dataframe has 16 districts.


In [ ]:
Create a map of Shanghai with districts superimposed on top.

In [79]:
# create map of Shanghai using latitude and longitude values
map_shanghai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, District in zip(Districts['Latitude'], Districts['Longitude'], Districts['District']):
    label = '{}'.format(District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffff00',
        fill_opacity=0.4,
        parse_html=False).add_to(map_shanghai) 
map_shanghai

## Explore Districts in Shanghai

**Let's get the venues that are in each distrcits of Shanghai within a radius of 500 meters.**

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
shanghai_venues = getNearbyVenues(names=Districts['District'],
                                   latitudes=Districts['Latitude'],
                                   longitudes=Districts['Longitude']
                                  )


黄浦区
徐汇区
长宁区
静安区
普陀区
虹口区
杨浦区
闵行区
宝山区
嘉定区
浦东新区
金山区
松江区
青浦区
奉贤区
崇明区


In [90]:
print(shanghai_venues.shape)
shanghai_venues.head()

(138, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,黄浦区,31.222771,121.490317,Shanghai Old West Gate International Youth Hos...,31.220857,121.487581,Hostel
1,黄浦区,31.222771,121.490317,东北四季饺子王 Dongbei Siji Jiaoziwang,31.221456,121.494210,Dumpling Restaurant
2,黄浦区,31.222771,121.490317,FamilyMart (全家),31.220681,121.494038,Convenience Store
3,黄浦区,31.222771,121.490317,隧道科技馆,31.222947,121.495267,Museum
4,徐汇区,31.179973,121.437520,Starbucks (星巴克),31.181764,121.437614,Coffee Shop


In [91]:
shanghai_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
嘉定区,6,6,6,6,6,6
奉贤区,2,2,2,2,2,2
宝山区,5,5,5,5,5,5
徐汇区,22,22,22,22,22,22
普陀区,1,1,1,1,1,1
杨浦区,4,4,4,4,4,4
松江区,4,4,4,4,4,4
浦东新区,1,1,1,1,1,1
虹口区,16,16,16,16,16,16


Let's find out how many unique categories can be curated from all the returned venues.

In [92]:
print('There are {} uniques categories.'.format(len(shanghai_venues['Venue Category'].unique())))

There are 67 uniques categories.


## Analyze Each District

In [147]:
# one hot encoding
shanghai_onehot = pd.get_dummies(shanghai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
shanghai_onehot['District'] = shanghai_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [shanghai_onehot.columns[-1]] + list(shanghai_onehot.columns[:-1])
shanghai_onehot = shanghai_onehot[fixed_columns]

shanghai_onehot.head()

,District,Asian Restaurant,Bagel Shop,Bakery,Beer Bar,Big Box Store,Burger Joint,Bus Station,Café,Chinese Restaurant,Coffee Shop,Confucian Temple,Convenience Store,Department Store,Dry Cleaner,Dumpling Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Garden,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lounge,Market,Metro Station,Mobile Phone Shop,Movie Theater,Museum,New American Restaurant,Noodle House,Park,Pizza Place,Playground,Plaza,Pool,Ramen Restaurant,Rock Climbing Spot,Sandwich Place,Shopping Mall,Soccer Field,Soccer Stadium,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Squash Court,Stadium,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,黄浦区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,黄浦区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,黄浦区,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,黄浦区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,徐汇区,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [148]:
shanghai_onehot.shape

(138, 68)

In [149]:
shanghai_grouped = shanghai_onehot.groupby('District').mean().reset_index()
shanghai_grouped

,District,Asian Restaurant,Bagel Shop,Bakery,Beer Bar,Big Box Store,Burger Joint,Bus Station,Café,Chinese Restaurant,Coffee Shop,Confucian Temple,Convenience Store,Department Store,Dry Cleaner,Dumpling Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Garden,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lounge,Market,Metro Station,Mobile Phone Shop,Movie Theater,Museum,New American Restaurant,Noodle House,Park,Pizza Place,Playground,Plaza,Pool,Ramen Restaurant,Rock Climbing Spot,Sandwich Place,Shopping Mall,Soccer Field,Soccer Stadium,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Squash Court,Stadium,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,嘉定区,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.166667,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.0,0.166667,0.166667,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000000,0.0000,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
1,奉贤区,0.5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000000,0.0000,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
2,宝山区,0.0,0.2,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000,0.00,0.000000,0.000000,0.0000,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.2,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
3,徐汇区,0.0,0.0,0.0,0.000000,0.045455,0.000000,0.045455,0.0,0.090909,0.090909,0.000000,0.045455,0.0000,0.00,0.000000,0.136364,0.045455,0.0,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.090909,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.090909,0.000,0.000,0.00,0.000000,0.000000,0.0000,0.0000,0.00,0.000,0.045455,0.045455,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.0,0.000000,0.045455,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
4,普陀区,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000000,0.0000,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
5,杨浦区,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.250000,0.0000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000000,0.2500,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.00

In [150]:
num_top_venues = 5

for hood in shanghai_grouped['District']:
    print("----"+hood+"----")
    temp = shanghai_grouped[shanghai_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----嘉定区----
              venue  freq
0            Garden  0.17
1     Grocery Store  0.17
2     Historic Site  0.17
3  Confucian Temple  0.17
4       Coffee Shop  0.17


----奉贤区----
              venue  freq
0  Asian Restaurant   0.5
1       Coffee Shop   0.5
2  Ramen Restaurant   0.0
3              Pool   0.0
4             Plaza   0.0


----宝山区----
                 venue  freq
0           Bagel Shop   0.2
1              Stadium   0.2
2  Sporting Goods Shop   0.2
3          Coffee Shop   0.2
4               Market   0.2


----徐汇区----
                  venue  freq
0  Fast Food Restaurant  0.14
1         Metro Station  0.09
2    Chinese Restaurant  0.09
3           Coffee Shop  0.09
4                 Hotel  0.09


----普陀区----
                venue  freq
0         Flea Market   1.0
1                Park   0.0
2  Rock Climbing Spot   0.0
3    Ramen Restaurant   0.0
4                Pool   0.0


----杨浦区----
                venue  freq
0                Park  0.25
1               Hotel  0.25


In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = shanghai_grouped['District']

for ind in np.arange(shanghai_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(shanghai_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,嘉定区,Garden,Japanese Restaurant,Grocery Store,Historic Site,Confucian Temple,Coffee Shop,Flea Market,Gym,Gym / Fitness Center,Vietnamese Restaurant
1,奉贤区,Asian Restaurant,Coffee Shop,Hostel,Garden,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Fish & Chips Shop
2,宝山区,Coffee Shop,Bagel Shop,Market,Stadium,Sporting Goods Shop,Vietnamese Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site
3,徐汇区,Fast Food Restaurant,Hotel,Metro Station,Chinese Restaurant,Coffee Shop,Ramen Restaurant,Soccer Stadium,Pool,Gym / Fitness Center,Rock Climbing Spot
4,普陀区,Flea Market,Vietnamese Restaurant,Fish & Chips Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hunan Restaurant,Hotpot Restaurant,Hotel Bar,Hotel
5,杨浦区,Chinese Restaurant,Park,Convenience Store,Hotel,Hostel,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Vietnamese Restaurant
6,松江区,Indian Restaurant,Dry Cleaner,Metro Station,Park,Vietnamese Restaurant,Historic Site,Grocery Store,Gym,Gym / Fitness Center,Hotel
7,浦东新区,Asian Restaurant,Fish & Chips Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hunan Restaurant,Hotpot Restaurant,Hotel Bar,Hotel,Hostel
8,虹口区,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Park,Fast Food Restaurant,Grocery Store,Department Store,Shopping Mall,Pizza Place,Sushi Restaurant
9,金山区,Hotel,Vietnamese Restaurant,Fish & Chips Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hunan Restaurant,Hotpot Restaurant,Hotel Bar,Hostel


## Cluster Districts of Shanghai

In [153]:
# set number of clusters
kclusters = 5

shanghai_grouped_clustering = shanghai_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(shanghai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 3, 1, 1, 4, 1, 2], dtype=int32)

In [154]:
# add clustering labels
districts_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

districts_merged = Districts

# merge districts_merged with districts to add latitude/longitude for each district
districts_merged = districts_merged.join(districts_venues_sorted.set_index('District'), on='District')

districts_merged.head() # check the last columns!

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,黄浦区,31.222771,121.490317,1.0,Hostel,Dumpling Restaurant,Convenience Store,Museum,Vietnamese Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hotel
1,徐汇区,31.179973,121.437520,1.0,Fast Food Restaurant,Hotel,Metro Station,Chinese Restaurant,Coffee Shop,Ramen Restaurant,Soccer Stadium,Pool,Gym / Fitness Center,Rock Climbing Spot
2,长宁区,31.218123,121.422200,1.0,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Café,Beer Bar,Fast Food Restaurant,Noodle House,Gym,Lounge,Convenience Store
3,静安区,31.229003,121.448224,1.0,Coffee Shop,Bakery,Gym / Fitness Center,Vietnamese Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Dumpling Restaurant,Historic Site,Hotel
4,普陀区,31.241701,121.392499,3.0,Flea Market,Vietnamese Restaurant,Fish & Chips Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hunan Restaurant,Hotpot Restaurant,Hotel Bar,Hotel


In [174]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(districts_merged['Latitude'], districts_merged['Longitude'], districts_merged['District'], districts_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float